<a href="https://colab.research.google.com/github/vlassner/dsml4220_lab10/blob/main/vl_lab10_agents_and_tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 10: A simple Agent with Tools

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sgeinitz/DSML4220/blob/main/lab10_agents_and_tools.ipynb)

In this lab we will use Ollama to create a simple agent armed with tools in order to help carry out tasks on our behalf. This notebook is based on the short blog posts/tutorials found [here](https://www.cohorte.co/blog/using-ollama-with-python-step-by-step-guide) and [here](https://towardsdatascience.com/ai-agents-from-zero-to-hero-part-1/).


### Lab 10 Assignment/Task
There are a few questions below that require some additional code to be written so that your agent can carry out other operations besides just addition.

Let's start out by setting up Ollama to run in Colab. If you run this notebook locally and already have Ollama running, then you can skip these steps.

In [ ]:
!pip install colab-xterm
%load_ext colabxterm

Next we'll start a terminal from within Colab. Once the terminal is running, you will need to run the two commands, but at separate times.

After you have run `%xterm` and the terminal opens, copy and paste the following line into the terminal and press enter:
* `curl https://ollama.ai/install.sh | sh`

After that has run, then start the Ollama server and have it run in the background by copying and pasting the following line into the terminal and pressing enter:
* ` ollama serve &`

(_Note: you may need to press enter one more time after `ollama serve` to see the terminal prompt again_)

Next we will pull the small (1B parameter) Llama3.2 model down by running the following in the terminal:
* `ollama pull llama3.2:1b`

In [ ]:
%xterm

Now that Ollama is running, we can get started. The only module/library needed for this is the ollama python module.

In [ ]:
!pip install ollama

In [ ]:
import ollama

Now, let's define a __tool__ for the agent/model to use.

In [ ]:
# Tool function to add two numbers
def add_two_numbers(a: int, b: int) -> int:
    return int(a) + int(b)

Next, let's set up the system prompt and an initial user prompt/question for the agent/model.

In [ ]:
# System prompt to inform the model about the tool is usage
system_message = {
    "role": "system",
    "content": "You are a helpful assistant. You can do math by calling a function 'add_two_numbers' if needed."
}

# A sample of user input asking a math question
user_message = {
    "role": "user",
    "content": "What is 90999999 + 10000001?"
}

messages = [system_message, user_message]
messages

Ask the agent/model to respond.

In [ ]:
# Ask llama3.2 to respond
response = ollama.chat(
    model='llama3.2:1b',
    messages=messages,
    tools=[add_two_numbers]
)

In [ ]:
response.message

In [ ]:
response.message.content

In [ ]:
# Check if the model called a function
if response.message.tool_calls:
    for tool_call in response.message.tool_calls:
        func_name = tool_call.function.name   # e.g., "add_two_numbers"
        args = tool_call.function.arguments   # e.g., {"a": 10, "b": 10}
        # If the function name matches and we have it in our tools, execute it:
        if func_name == "add_two_numbers":
            result = add_two_numbers(**args)
            print("Function output:", result)




---

### Q1: Does the above output look correct? Does it look like the sum of the numbers 90999999 and 10000001? Why is it not correct?

(Hint: there is nothing wrong with the model/agent here, but rather the tool implementation; namely, Python's [type hints](https://docs.python.org/3/library/typing.html) are not a guarantee that the correct/intended data type is used, so you may need to add some type casting inside of the function `add_two_numbers`)

The original response concatinates the values together to be 0900000010000001. After the function's values has been type cast, the answer is 101000000.

---

In [ ]:
# Complete the agent's tool call and allow the model to use output to formulate an answer
""" (Continuing from previous code) """
available_functions = {"add_two_numbers": add_two_numbers, "multiply_two_numbers": multiply_two_numbers}

""" System prompt to inform the model about the tool is usage """

""" Model's initial response after possibly invoking the tool """
assistant_reply = response.message.content
print("Assistant (initial):", assistant_reply)

""" If a tool was called, handle it """
for tool_call in (response.message.tool_calls or []):
    func = available_functions.get(tool_call.function.name)
    if func:
        result = func(**tool_call.function.arguments)
        # Provide the result back to the model in a follow-up message
        messages.append({"role": "assistant", "content": f"The result is {result}."})
        messages.append({"role": "user", "content": "Can you summarize and state the results you found?"})
        follow_up = ollama.chat(model='llama3.2:1b', messages=messages)
        print("Assistant (final):", follow_up.message.content)

---

### Q2: Try running the code cell below. Does it return the expect result? If note, then add/modify the necessary code to allow Llama3.2 to use its  multiplication tool. Then rerun your code cell below; now did it output the expected result?

The first response was the model saying it could not calculate the result because the function was incomplete. After fixing the function, it displayed a step by step process of how to multiply the numbers together.

---

In [ ]:
# Implement a multiplication function by replacing the `pass` statement below with the correct return statement
def multiply_two_numbers(a: int, b: int) -> int:
    return int(a) * int(b)


""" System prompt to inform the model about the tool is usage """
system_message = {
    "role": "system",
    "content": "You are a helpful assistant. You can do addition by calling the function 'add_two_numbers' or multiplication by calling the function 'multiply_two_numbers'."
}
# User asks a question that involves a calculation
user_message = {
    "role": "user",
    "content": "What is 10001 times 6?"
}

messages = [system_message, user_message]

response = ollama.chat(
    model='llama3.2:1b',
    messages=messages,
    tools=[add_two_numbers, multiply_two_numbers]  # pass the actual function object as a tool
)

# Model's initial reponse after (hopefully) calling the tool
assistant_reply = response.message.content
print("Assistant (initial):", assistant_reply)

# If a tool was called, then handle it
available_functions = {"add_two_numbers": add_two_numbers, "multiply_two_numbers": multiply_two_numbers}
for tool_call in (response.message.tool_calls or []):
    func = available_functions.get(tool_call.function.name)
    if func:
        result = func(**tool_call.function.arguments)
        # Provide the result back to the model in a follow-up message
        messages.append({"role": "assistant", "content": f"The result is {result}."})
        messages.append({"role": "user", "content": "Can you summarize and state the results you found?"})
        follow_up = ollama.chat(model='llama3.2:1b', messages=messages)
        print("Assistant (final):", follow_up.message.content)

In [ ]:
follow_up.message

Next let's equip our agent to retrieve external information, which will require a few more tools to be able to search the web.

In [ ]:
!pip install langchain_community

In [ ]:
!pip install -U duckduckgo-search

In [ ]:
from langchain_community.tools import DuckDuckGoSearchResults


def search_web(query: str) -> str:
  return DuckDuckGoSearchResults(backend="news").run(query)

tool_search_web = {'type':'function', 'function':{
  'name': 'search_web',
  'description': 'Search the web',
  'parameters': {'type': 'object',
                'required': ['query'],
                'properties': {
                    'query': {'type':'str', 'description':'the topic or subject to search on the web'},
}}}}

# Quickly test and see what a general web news search for Los Angeles yields
search_web(query="Los Angeles")

In [ ]:
def search_ys(query: str) -> str:
  engine = DuckDuckGoSearchResults(backend="news")
  return engine.run(f"site:sports.yahoo.com {query}")

tool_search_ys = {'type':'function', 'function':{
  'name': 'search_ys',
  'description': 'Search for sports news',
  'parameters': {'type': 'object',
                'required': ['query'],
                'properties': {
                    'query': {'type':'str', 'description':'the sport, sports team, or subject to search'},
}}}}

# Quickly test and see what a search for Los Angeles in the sports section of the news yields
search_ys(query="Los Angeles")

In [ ]:
system_message = {
    "role": "system",
    "content": "You are a helpful assistant with access to tools for search the web for current news and events."
    }
user_message = {
    "role": "user",
    "content": "Tell me about sports in the city of Denver." # YOU WILL CHANGE THIS QUESTION, SEE Q3 BELOW
}
messages = [system_message, user_message]

In [ ]:
messages

In [ ]:
response = ollama.chat(
  model="llama3.2:1b",
  tools=[tool_search_web, tool_search_ys],
  messages=messages
)
response

In [ ]:
# Model's initial reponse after (hopefully) calling the tool
assistant_reply = response.message.content
print("Assistant (initial):", assistant_reply)

# If a tool was called, then handle it
available_functions = {'search_web':search_web, 'search_ys':search_ys}
for tool_call in (response.message.tool_calls or []):
    func = available_functions.get(tool_call.function.name)
    if func:
        result = func(**tool_call.function.arguments)
        # Provide the result back to the model in a follow-up message
        messages.append({"role": "assistant", "content": f"The result is {result}."})
        messages.append({"role": "user", "content": "Can you summarize and state the results you found?"})
        follow_up = ollama.chat(model='llama3.2:1b', messages=messages)
        print("Assistant (final):", follow_up.message.content)

---

### Q3: The question above currently asks about Denver, but change the question to include a word or reference to sports. Does the agent use the correct tool based on your prompt/question? Be sure to also run the code cells above with your modified promp/question.

The first response only mentioned broncos related events and after adding the word sport to it, it mentioned NWSL and the Denver nuggets. It does use the correct tools to search Duck Duck Go for recent news. Rewording the message does drastically change the output like putting sports before or after the city of denver part.

---